<a href="https://colab.research.google.com/github/jsvon/NLP-projects-/blob/main/Agentic_RAG_query_reformulation_and_self_query!5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reminder: Retrieval-Augmented-Generation (RAG) is “using an LLM to answer a user query, but basing the answer on information retrieved from a knowledge base”. It has many advantages over using a vanilla or fine-tuned LLM: to name a few, it allows to ground the answer on true facts and reduce confabulations, it allows to provide the LLM with domain-specific knowledge, and it allows fine-grained control of access to information from the knowledge base.


RAG agent: very simply, an agent armed with a retriever tool!

In [1]:
!pip install pandas langchain langchain-community sentence-transformers faiss-cpu smolagents --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!pip install datasets
import datasets

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.2 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

In [6]:
#We use LangChain for its excellent vector database utilities.

from tqdm import tqdm
from transformers import AutoTokenizer
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

source_docs = [
   Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]}) for doc in knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained("thenlper/gte-small"),
    chunk_size=100,
    chunk_overlap=20,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", " ", ""]
)

# split docs and only keep unique ones

print("Splitting documents ...")
docs_processed = []
unique_texts = {}
for doc in tqdm(source_docs):
  new_docs = text_splitter.split_documents([doc])
  for new_doc in new_docs:
    if new_doc.page_content in unique_texts:
      continue
    unique_texts[new_doc.page_content] = True
    docs_processed.append(new_doc)
print ("Embedding documents ... This should take a few minutes (5 minutes on MacBook with M1 Pro, but you dont own one, do you?")
embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")
vectordb = FAISS.from_documents(
    documents=docs_processed,
    embedding=embedding_model,
    distance_strategy=DistanceStrategy.COSINE
)

Splitting documents ...


100%|██████████| 2647/2647 [03:31<00:00, 12.52it/s]


Embedding documents ... This should take a few minutes (5 minutes on MacBook with M1 Pro, but you dont own one, do you?


In [7]:
from smolagents import Tool
from langchain_core.vectorstores import VectorStore

class RetrieverTool(Tool):
  name = "retriever"
  description = "Using semantic similarity, retrieves some documents from the knowledge base that have the closest embeddings to the input query."
  inputs = {
      "query":{
          "type": "string",
          "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
      }
  }

  output_type = "string"

  def __init__(self, vectordb: VectorStore, **kwargs):
    super().__init__(**kwargs)
    self.vectordb = vectordb

  def forward (self, query: str) -> str:
    assert isinstance(query, str), "Your search query must be a string"

    docs = self.vectordb.similarity_search(
        query,
        k=7,

    )

    return "\nRetrieved document:\n" + "".join(
        [f"==== Document {str(i)} ====\n" + doc.page_content for i, doc in enumerate(docs)]
    )

In [13]:
from smolagents import HfApiModel, ToolCallingAgent

model = HfApiModel(model_id='https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud')

retriever_tool = RetrieverTool(vectordb)
agent = ToolCallingAgent(tools=[retriever_tool], model=model)

In [14]:
agent_output = agent.run("How can I push a model to the Hub")

print("Final output:")
print (agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How can I push a model to the Hub                                                                               │
│                                                                                                                 │
╰─ HfApiModel - https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud ───────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'retriever' with arguments: {'query': 'How to push a model to the Hugging Face Hub?'}             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Retrieved document:
==== Document 0 ====
- `push_to_hub`: a parameter to push the final trained model to the Hugging Face Hub.==== Document 1 ====
Finally, if you want, you can push your model up to the hub. Here, we'll push it up if you specified 
`push_to_hub=True` in the training configuration. Note that in order to push to hub, you'll have to have git-lfs 
installed and be logged into your Hugging Face account (which can be done via `huggingface-cli login`).==== 
Document 2 ====
the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model).==== Document
3 ====
### Pushing the model to HF Hub

With the `peft` model, it is also very easy to push a model the Hugging Face Hub. Below, we demonstrate how it 
works. It is assumed that you have a valid Hugging Face account and are logged in:==== Document 4 ====
`push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of==== Document 5 
====
|Trainers `push_to-hub` 
functionality](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_h
ub) so that your model will be automatically uploaded to the Hub.==== Document 6 ====
## Share Model Cards

If you're authenticated with the Hugging Face Hub (either by using `huggingface-cli login` or |`login`]), you can 
push cards to the Hub by simply calling |`ModelCard.push_to_hub`]. Let's take a look at how to do that...

[Step 1: Duration 3.24 seconds| Input tokens: 1,201 | Output tokens: 30]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'To push a model to the Hugging Face Hub, you need to   │
│ set `push_to_hub=True` in your training configuration or model pushing function call. Ensure you have git-lfs   │
│ installed and are logged into your Hugging Face account using `huggingface-cli login`. '}                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: To push a model to the Hugging Face Hub, you need to set `push_to_hub=True` in your training 
configuration or model pushing function call. Ensure you have git-lfs installed and are logged into your Hugging 
Face account using `huggingface-cli login`. 

[Step 2: Duration 2.19 seconds| Input tokens: 2,828 | Output tokens: 101]

Final output:
To push a model to the Hugging Face Hub, you need to set `push_to_hub=True` in your training configuration or model pushing function call. Ensure you have git-lfs installed and are logged into your Hugging Face account using `huggingface-cli login`. 
